In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from dask.diagnostics import ProgressBar
from itertools import product
from joblib import Parallel, delayed
from collections import defaultdict

import common
import vcm
import fv3viz

In [2]:
@common.memoize_xarray_out
def get_global_avg(group, field):
    data = common.open_group(group)
    # data is already on pressure levels
    return vcm.weighted_average(data[field], data.area).load().to_dataset()

Make it faster

In [3]:
base = "combined-zcemu-monthly-init-m{init}-30d-v1-{run_type}"
fields = [
    "air_temperature",
    "specific_humidity",
    "cloud_water_mixing_ratio"
]
run_types = ["online", "offline"]
inits = [f"{i:02d}" for i in range(1, 13)]

In [4]:

args = list(product(inits, run_types, fields))
jobs = []

for init, run_type, field in args:
    group = base.format(init=init, run_type=run_type)
    jobs.append(delayed(get_global_avg)(group, field))

    

In [5]:
results = Parallel(n_jobs=12)(jobs)

In [6]:
results[0]

<xarray.Dataset>
Dimensions:          (z: 79, time: 240)
Coordinates:
  * z                (z) float64 4.514 8.301 12.45 16.74 ... 989.5 994.4 998.3
  * time             (time) object 2016-01-31 03:00:00 ... 2016-03-01 00:00:00
Data variables:
    air_temperature  (time, z) float64 ...

In [7]:
args[0]

('01', 'online', 'air_temperature')

In [8]:
def gather(field_key, args, results):
    out = defaultdict(list)

    for i, ds in enumerate(results):
        month, run_type, field = args[i]
        if field == field_key:
            out[run_type].append((month, ds))
    
    return out

In [10]:
cloud = gather("cloud_water_mixing_ratio", args, results)

KeyError: 'online'

In [24]:
args[0]

('01', 'online', 'air_temperature')